## Import lib

In [39]:
#!!!
import os
import pandas as pd
import sys
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random

from sklearn import preprocessing
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold, train_test_split
from sklearn.metrics import f1_score, roc_auc_score, recall_score, precision_score, confusion_matrix,accuracy_score,classification_report
import lightgbm as lgb
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
# from sklearn.datasets import load_files
# sys.path.append('..') 
# import helpers.data_mining_helpers as dmh

# import nltk
# nltk.download('punkt')
# import seaborn as sns
# from sklearn.decomposition import PCA
# from sklearn import preprocessing, metrics, decomposition, pipeline, dummy
# from sklearn.preprocessing import binarize
# from sklearn.metrics.pairwise import cosine_similarity



## Data input

In [74]:
#!!!
train_file_path = '..\\training.csv'
test_file_path = '..\\public_processed.csv'
test_file_answer_path = '..\\public.csv'
test_private_file_path = '..\\private_1_processed.csv'

# train_file_path = 'training.csv'
# test_file_path = 'public_processed.csv'
# test_file_answer_path = 'public.csv'
# test_private_file_path = 'private_1_processed.csv'

df = pd.read_csv(train_file_path)
testing_df = pd.read_csv(test_file_path)
testing_answer_df = pd.read_csv(test_file_answer_path)
testing_private_df = pd.read_csv(test_private_file_path)

df.shape : (8688526, 26)  |||  test_df.shape : (600182, 25) ||| testing_answer_df.shape : (609040, 26) ||| testing_private_df.shape : (754139, 25)

In [75]:
#!!!
# training_df = df.copy()
# test_df = testing_df.copy()
training_df = df
test_df = testing_df.copy()
y = training_df['label']
training_df = training_df.drop(['label','txkey'], axis=1)
test_df = test_df.drop(['txkey'], axis=1)
# testing_answer_df = testing_answer_df.drop(['txkey'], axis=1)
pridict = pd.DataFrame(testing_private_df['txkey'] )
testing_private_df = testing_private_df.drop(['txkey'], axis=1)

In [78]:
#!!!
train_num = training_df.shape[0]
test_num = test_df.shape[0]
private_num = test_num + train_num
allData = pd.concat([training_df ,test_df, testing_private_df], axis=0)


In [79]:
#!!!

# print(df.shape)
# print(test_df.shape)

# Rename the columns to match the previous code
# df.columns = ['txkey', 'locdt', 'loctm', 'chid', 'cano', 'locdt']
# test_df.columns = ['sentence', 'score', 'provider']

# Combine the two dataframes
# X = test_df
# display(test_df.head())
# display(df.head())
# Randomly select 3000 rows and drop duplicates and missing values
# X = X.sample(n=3000, random_state=42, replace=False)
# X.dropna(inplace=True)
# X.drop_duplicates(inplace=True)
# display(X)


label_encoder_chid = LabelEncoder()
label_encoder_cano = LabelEncoder()
label_encoder_mchno = LabelEncoder()
label_encoder_acqic = LabelEncoder()
label_encoder_etymd = LabelEncoder()
label_encoder_mcc = LabelEncoder()
label_encoder_stocn = LabelEncoder()
label_encoder_scity = LabelEncoder()
label_encoder_stscd = LabelEncoder()
label_encoder_hcefg = LabelEncoder()
label_encoder_csmcu = LabelEncoder()

allData['chid'] = label_encoder_chid.fit_transform(allData['chid'])
allData['cano'] = label_encoder_cano.fit_transform(allData['cano'])
allData['mchno'] = label_encoder_mchno.fit_transform(allData['mchno'])
allData['acqic'] = label_encoder_acqic.fit_transform(allData['acqic'])
allData['etymd']= label_encoder_etymd.fit_transform(allData['etymd'])
allData['mcc']= label_encoder_mcc.fit_transform(allData['mcc'])
allData['stocn']= label_encoder_stocn.fit_transform(allData['stocn'])
allData['scity']= label_encoder_scity.fit_transform(allData['scity'])
allData['stscd']= label_encoder_stscd.fit_transform(allData['stscd'])
allData['hcefg']= label_encoder_hcefg.fit_transform(allData['hcefg'])
allData['csmcu']= label_encoder_csmcu.fit_transform(allData['csmcu'])


allData['chid'].astype("category")
allData['cano'].astype("category")
allData['mchno'].astype("category")
allData['acqic'].astype("category")
allData['etymd'].astype("category")
allData['mcc'].astype("category")
allData['stocn'].astype("category")
allData['scity'].astype("category")
allData['stscd'].astype("category")
allData['hcefg'].astype("category")
allData['csmcu'].astype("category")

allData['contp'].astype("category")
allData['etymd'].astype("category")
allData['ecfg'].astype("category")
allData['insfg'].astype("category")
allData['bnsfg'].astype("category")
allData['ovrlt'].astype("category")
allData['flbmk'].astype("category")
allData['flg_3dsmk'].astype("category")



0         0
1         0
2         0
3         0
4         0
         ..
754134    0
754135    0
754136    0
754137    1
754138    0
Name: flg_3dsmk, Length: 10042847, dtype: category
Categories (2, int64): [0, 1]

In [80]:
#!!!
training_df = allData[:train_num]
test_df = allData[train_num:private_num]
testing_private_df = allData[private_num:]

In [49]:
print(train_num, test_num, private_num)

8688526 600182 1200364


In [52]:
display(training_df)
display(test_df)
display(testing_private_df)

,locdt,loctm,chid,cano,contp,etymd,mchno,acqic,mcc,conam,...,flam1,stocn,scity,stscd,ovrlt,flbmk,hcefg,csmcu,csmam,flg_3dsmk
0,46,101812,256109,154065,5,1,138119,4871,351,1524.0,...,1529,0,11874,5,0,0,6,68,1529,0
1,17,85509,297980,350989,5,7,45877,1881,258,101.0,...,101,0,11874,5,0,0,8,68,101,0
2,44,212641,425640,76572,5,4,62433,4871,264,117.0,...,116,0,11874,5,0,0,6,68,116,0
3,42,102702,168531,167846,5,1,138119,1844,351,1676.0,...,1683,0,11874,5,0,0,6,68,1683,0
4,31,185737,493852,407593,5,5,55077,1881,382,14.0,...,14,0,11874,5,0,0,6,68,14,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688521,14,84737,325181,174672,5,4,41109,3193,258,49.0,...,49,0,11874,5,0,0,0,68,49,0
8688522,23,105923,439282,104273,5,4,167076,4871,252,4020.0,...,4018,0,11874,5,0,0,6,68,4018,0
8688523,23,141030,257694,103443,5,4,65068,1613,248,315.0,...,314,0,8472,5,0,0,6,68,314,0
8688524,10,120209,57843,630419,5,5,136008,1881,248,226.0,...,225,0,11874,5,0,0,6,68,225,0


,locdt,loctm,chid,cano,contp,etymd,mchno,acqic,mcc,conam,...,flam1,stocn,scity,stscd,ovrlt,flbmk,hcefg,csmcu,csmam,flg_3dsmk
0,56,105157,414123,389764,4,1,95079,1425,248,1353.00,...,1349,0,11874,5,0,0,6,68,1349,0
1,56,185520,350019,396507,5,7,150662,2409,296,305.00,...,306,0,10068,5,0,0,6,68,306,0
2,59,102312,273500,168466,5,1,105697,5242,466,20.00,...,20,0,11874,5,0,0,11,82,0,0
3,58,230138,481778,381976,5,5,128957,3802,248,8371.00,...,8339,40,9251,5,0,0,6,68,8339,0
4,59,101937,77056,294786,5,5,14552,4659,194,163.69,...,163,120,11109,5,0,0,6,66,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600177,56,93445,74460,84881,4,1,145268,664,208,96.00,...,96,0,12143,5,0,0,6,68,96,0
600178,56,145420,427331,34935,5,5,144308,1881,351,2014.00,...,2017,0,11874,5,0,0,6,68,2017,0
600179,58,3400,153100,178100,5,1,47446,4456,382,33.00,...,33,120,7624,5,0,0,6,68,33,0
600180,59,134901,389252,195847,5,4,1484,4871,264,1800.00,...,1796,0,11874,5,0,0,6,68,1796,0


,locdt,loctm,chid,cano,contp,etymd,mchno,acqic,mcc,conam,...,flam1,stocn,scity,stscd,ovrlt,flbmk,hcefg,csmcu,csmam,flg_3dsmk
0,64,63017,484998,269010,5,4,141954,4871,264,100.0,...,100,0,11874,5,0,0,6,68,100,0
1,60,113414,45092,197043,5,5,158522,1881,252,222.0,...,222,0,11874,5,0,0,6,68,222,0
2,64,101801,209476,539214,4,1,145268,4504,208,259.0,...,260,0,12143,5,0,0,6,68,260,0
3,61,4705,56593,619224,4,7,13085,8077,324,3111.0,...,3098,0,11874,5,0,0,6,68,3098,0
4,60,190432,419897,433534,2,1,59776,4871,265,992.0,...,996,0,11874,5,0,0,6,68,996,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754134,62,212026,103615,54550,5,5,49315,2409,296,209.0,...,209,0,12142,5,0,0,6,68,209,0
754135,64,170058,232916,39574,5,5,79991,1881,252,189.0,...,188,0,11874,5,0,0,6,68,188,0
754136,62,182252,82755,308568,5,4,99169,6360,248,9100.0,...,9091,0,11874,5,0,0,0,68,9091,0
754137,61,122044,406705,323914,5,5,49315,2409,296,86.0,...,86,0,12142,5,0,0,6,68,86,1


## Data preprocess.

type:  
type0 : txkeys     
type1 (times) : locdt loctm        
type2 (money) : conam flam1 ovrlt csmcu csmam      
type3 (person) : chid cano     
type4 (places) : mchno stocn scity     
type5 (other trade types) : contp etymd acqic mcc ecfg insfg iterm bnsfg stscd flbmk hcefg flg_3dsmk       


analysis part.

In [81]:
#!!!




fraud_df = training_df[df['label'] == 1]
non_fraud_df = training_df[df['label'] == 0]
# print(fraud_df.shape)
# print(non_fraud_df.shape)
# print(training_df.shape)


In [6]:
# each feature should use ifts own label encoder so that the unseen value can be handled
# try:
#     print(label_encoder.transform([-1]))
# except ValueError:
#     print("unseen value")


In [7]:
# training_df.drop(['txkey'], axis=1, inplace=True)

In [11]:
# print(fraud_df[col].shape[0]/ training_df[col].shape[0])
# print(non_fraud_df[col].shape[0]/ training_df[col].shape[0])
for col in training_df.columns:  
    print(f"col: {col}")
    # print(f"nunique: {training_df[col].nunique()}")
    # print(f"unique values: {training_df[col].unique()}")
    # print(f"fraud nunique: {fraud_df[col].nunique()}")
    # print(f"fraud unique values: {fraud_df[col].unique()}")
    # print(f"non fraud nunique: {non_fraud_df[col].nunique()}")
    # print(f"non fraud unique values: {non_fraud_df[col].unique()}")
    # print(fraud_df[col].nunique()/ training_df[col].nunique())
    # print(non_fraud_df[col].nunique()/ training_df[col].nunique()) 
    # unique_values = fraud_df[col].unique()
    # print(f"unique values: {unique_values}")

    #average_ratio
    # average_ratio = 0
    # for value in unique_values:
    #     average_ratio += fraud_df[col][fraud_df[col] == value].shape[0]/ training_df[col][training_df[col] == value].shape[0]
    # average_ratio /= len(unique_values)
    # print(f"average_ratio: {average_ratio}")

    #real_ratio
    # unique_values = fraud_df[col].unique()
    # numerator = np.sum(np.isin(fraud_df[col], unique_values))
    # denominator = np.sum(np.isin(training_df[col], unique_values))
    # average_ratio = numerator/denominator
    # print(f"real_ratio: {average_ratio}")

    #max and min ratio top 10 and plot
    # unique_values = fraud_df[col].unique()
    # ratio_list = []
    # for value in unique_values:
    #     ratio = fraud_df[col][fraud_df[col] == value].shape[0]/ training_df[col][training_df[col] == value].shape[0]
    #     ratio_list.append(ratio)
    # ratio_list.sort(reverse=True)
    # if len(ratio_list) > 10:
    #     print(f"max ratio top 10: {ratio_list[:10]}")
    #     plt.figure(figsize=(10, 5))
    #     plt.bar(np.arange(1, 11), ratio_list[:10])
    #     plt.title(f"Max Ratio of Fraud Transactions for {col}")
    #     plt.xlabel("Unique Values")
    #     plt.ylabel("Max Ratio of Fraud Transactions")
    #     plt.show()
    #     print(f"min ratio top 10: {ratio_list[-10:]}")
    #     plt.figure(figsize=(10, 5))
    #     plt.bar(np.arange(1, 11), ratio_list[-10:])
    #     plt.title(f"Min Ratio of Fraud Transactions for {col}")
    #     plt.xlabel("Unique Values")
    #     plt.ylabel("Min Ratio of Fraud Transactions")
    #     plt.show()
    # else:
    #     print(f"max ratio: {ratio_list}")
    #     plt.figure(figsize=(10, 5))
    #     plt.bar(unique_values, ratio_list)
    #     plt.title(f"Max Ratio of Fraud Transactions for {col}")
    #     plt.xlabel("Unique Values")
    #     plt.ylabel("Max Ratio of Fraud Transactions")
    #     plt.show()
    
    # show the max and the min value
    # print(f"max value: {training_df[col].max()}")
    # print(f"min value: {training_df[col].min()}")

    #show the num of ratio 1-0.9 , 0.9-0.8, 0.8-0.7, 0.7-0.6, 0.6-0.5, 0.5-0.4, 0.4-0.3, 0.3-0.2, 0.2-0.1, 0.1-0
    unique_values = fraud_df[col].unique()
    ratio_10_9_num = 0
    ratio_9_8_num = 0
    ratio_8_7_num = 0
    ratio_7_6_num = 0
    ratio_6_5_num = 0
    ratio_5_4_num = 0
    ratio_4_3_num = 0
    ratio_3_2_num = 0
    ratio_2_1_num = 0
    ratio_1_0_num = 0
    for value in unique_values:
        ratio = fraud_df[col][fraud_df[col] == value].shape[0]/ training_df[col][training_df[col] == value].shape[0]
        if ratio >= 0.9:
            ratio_10_9_num += fraud_df[col][fraud_df[col] == value].shape[0]
        elif ratio >= 0.8:
            ratio_9_8_num += fraud_df[col][fraud_df[col] == value].shape[0]
        elif ratio >= 0.7:
            ratio_8_7_num += fraud_df[col][fraud_df[col] == value].shape[0]
        elif ratio >= 0.6:
            ratio_7_6_num += fraud_df[col][fraud_df[col] == value].shape[0]
        elif ratio >= 0.5:
            ratio_6_5_num += fraud_df[col][fraud_df[col] == value].shape[0]
        elif ratio >= 0.4:
            ratio_5_4_num += fraud_df[col][fraud_df[col] == value].shape[0]
        elif ratio >= 0.3:
            ratio_4_3_num += fraud_df[col][fraud_df[col] == value].shape[0]
        elif ratio >= 0.2:
            ratio_3_2_num += fraud_df[col][fraud_df[col] == value].shape[0]
        elif ratio >= 0.1:
            ratio_2_1_num += fraud_df[col][fraud_df[col] == value].shape[0]
        else:
            ratio_1_0_num += fraud_df[col][fraud_df[col] == value].shape[0]
    print(f"ratio_10_9_num: {ratio_10_9_num}, ratio: {ratio_10_9_num/ fraud_df.shape[0]}")
    print(f"ratio_9_8_num: {ratio_9_8_num}, ratio: {ratio_9_8_num/ fraud_df.shape[0]}")
    print(f"ratio_8_7_num: {ratio_8_7_num}, ratio: {ratio_8_7_num/ fraud_df.shape[0]}")
    print(f"ratio_7_6_num: {ratio_7_6_num}, ratio: {ratio_7_6_num/ fraud_df.shape[0]}")
    print(f"ratio_6_5_num: {ratio_6_5_num}, ratio: {ratio_6_5_num/ fraud_df.shape[0]}")
    print(f"ratio_5_4_num: {ratio_5_4_num}, ratio: {ratio_5_4_num/ fraud_df.shape[0]}")
    print(f"ratio_4_3_num: {ratio_4_3_num}, ratio: {ratio_4_3_num/ fraud_df.shape[0]}")
    print(f"ratio_3_2_num: {ratio_3_2_num}, ratio: {ratio_3_2_num/ fraud_df.shape[0]}")
    print(f"ratio_2_1_num: {ratio_2_1_num}, ratio: {ratio_2_1_num/ fraud_df.shape[0]}")
    print(f"ratio_1_0_num: {ratio_1_0_num}, ratio: {ratio_1_0_num/ fraud_df.shape[0]}")
        
    
    print('\n')


col: locdt
ratio_10_9_num: 0, ratio: 0.0
ratio_9_8_num: 0, ratio: 0.0
ratio_8_7_num: 0, ratio: 0.0
ratio_7_6_num: 0, ratio: 0.0
ratio_6_5_num: 0, ratio: 0.0
ratio_5_4_num: 0, ratio: 0.0
ratio_4_3_num: 0, ratio: 0.0
ratio_3_2_num: 0, ratio: 0.0
ratio_2_1_num: 0, ratio: 0.0
ratio_1_0_num: 32029, ratio: 1.0


col: loctm
ratio_10_9_num: 0, ratio: 0.0
ratio_9_8_num: 0, ratio: 0.0
ratio_8_7_num: 0, ratio: 0.0
ratio_7_6_num: 0, ratio: 0.0
ratio_6_5_num: 1, ratio: 3.12217053295451e-05
ratio_5_4_num: 7, ratio: 0.0002185519373068157
ratio_4_3_num: 45, ratio: 0.0014049767398295295
ratio_3_2_num: 286, ratio: 0.008929407724249898
ratio_2_1_num: 1567, ratio: 0.048924412251397174
ratio_1_0_num: 30123, ratio: 0.940491429641887


col: chid
ratio_10_9_num: 27245, ratio: 0.8506353617034562
ratio_9_8_num: 301, ratio: 0.009397733304193075
ratio_8_7_num: 117, ratio: 0.0036529395235567765
ratio_7_6_num: 406, ratio: 0.01267601236379531
ratio_6_5_num: 348, ratio: 0.010865153454681694
ratio_5_4_num: 201, ratio:

Average ratio:  all unique value in fraud_df and average (amount in fraud_df / amount in training_df) for each feature(column)   
col: label      

---
col: cano       
col: chid       
col: scity      
col: mchno      
col: conam      

---
col: acqic      
col: stocn      
col: csmcu       
col: stscd      
col: flam1      
col: hcefg      
col: mcc        

---
col: csmam      
col: loctm      

---
col: locdt      
col: contp      
col: etymd      
col: ecfg       
col: insfg     
col: iterm      
col: bnsfg     
col: ovrlt   
col: flbmk   
col: flg_3dsmk 

Do feature preprocessing need to do three data part. 
1. training data. : training_df
2. testing data.(pulbic data) : test_df
3. predict data. : testing_private_df

In [95]:
#!!!
temp_training = training_df.copy()
temp_testing = test_df.copy()
temp_prediction = testing_private_df.copy()



In [64]:
training_df = temp_training.copy()
test_df = temp_testing.copy()
testing_private_df = temp_prediction.copy()

Create feature.

In [83]:
#Create a new column : if the ratio of cano value is larger than 0.9, then 1, else 0 xx
#Create a new column : the ratio of cano value
#cano_ratio
training_df.loc[:, 'cano_ratio'] = 0
test_df.loc[:, 'cano_ratio'] = 0
testing_private_df.loc[:, 'cano_ratio'] = 0
cano_ratio_map = {}
for value in fraud_df['cano'].unique():
    ratio = round(fraud_df['cano'][fraud_df['cano'] == value].shape[0]/ training_df['cano'][training_df['cano'] == value].shape[0],2) 
    cano_ratio_map[value] = ratio*100
training_df.loc[:, 'cano_ratio'] = training_df['cano'].map(cano_ratio_map)
test_df.loc[:, 'cano_ratio'] = test_df['cano'].map(cano_ratio_map)
testing_private_df.loc[:, 'cano_ratio'] = testing_private_df['cano'].map(cano_ratio_map)
training_df.loc[training_df['cano_ratio'].isna(), 'cano_ratio'] = 0
test_df.loc[test_df['cano_ratio'].isna(), 'cano_ratio'] = 0
testing_private_df.loc[testing_private_df['cano_ratio'].isna(), 'cano_ratio'] = 0
training_df[ 'cano_ratio'].astype("category")
test_df[ 'cano_ratio'].astype("category")
testing_private_df[ 'cano_ratio'].astype("category")



# display(training_df)
# display(training_df['cano_ratio'][training_df['cano'] == value])

# #Create a new column : the ratio of chid value
#chid_ratio
training_df.loc[:, 'chid_ratio'] = 0
test_df.loc[:, 'chid_ratio'] = 0
testing_private_df.loc[:, 'chid_ratio'] = 0
chid_ratio_map = {}
for value in fraud_df['chid'].unique():
    ratio = round(fraud_df['chid'][fraud_df['chid'] == value].shape[0]/ training_df['chid'][training_df['chid'] == value].shape[0],2) 
    chid_ratio_map[value] = ratio*100
training_df.loc[:, 'chid_ratio'] = training_df['chid'].map(chid_ratio_map)
test_df.loc[:, 'chid_ratio'] = test_df['chid'].map(chid_ratio_map)
testing_private_df.loc[:, 'chid_ratio'] = testing_private_df['chid'].map(chid_ratio_map)
training_df.loc[training_df['chid_ratio'].isna(), 'chid_ratio'] = 0
test_df.loc[test_df['chid_ratio'].isna(), 'chid_ratio'] = 0
testing_private_df.loc[testing_private_df['chid_ratio'].isna(), 'chid_ratio'] = 0
training_df[ 'chid_ratio'].astype("category")
test_df[ 'chid_ratio'].astype("category")
testing_private_df[ 'chid_ratio'].astype("category")


C:\Users\lo\AppData\Local\Temp\ipykernel_22796\2337224500.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_df.loc[:, 'cano_ratio'] = 0
C:\Users\lo\AppData\Local\Temp\ipykernel_22796\2337224500.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.loc[:, 'cano_ratio'] = 0
C:\Users\lo\AppData\Local\Temp\ipykernel_22796\2337224500.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
754134    0.0
754135    0.0
754136    0.0
754137    0.0
754138    0.0
Name: chid_ratio, Length: 754139, dtype: category
Categories (60, float64): [0.0, 1.0, 2.0, 3.0, ..., 88.0, 89.0, 96.0, 100.0]

In [93]:
# #New column (cano_ratio_bool):if the value of cano_ratio is > 0.9, then 1, else 0

#cano_ratio_bool
training_df.loc[:, 'cano_ratio_bool'] = 0
test_df.loc[:, 'cano_ratio_bool'] = 0
testing_private_df.loc[:, 'cano_ratio_bool'] = 0
training_df.loc[training_df['cano_ratio'] >= 90, 'cano_ratio_bool'] = 1
test_df.loc[test_df['cano_ratio'] >= 90, 'cano_ratio_bool'] = 1
testing_private_df.loc[testing_private_df['cano_ratio'] >= 90, 'cano_ratio_bool'] = 1
training_df[ 'cano_ratio_bool'].astype("category")
test_df[ 'cano_ratio_bool'].astype("category")
testing_private_df[ 'cano_ratio_bool'].astype("category")
#chid_ratio_bool
training_df.loc[:, 'chid_ratio_bool'] = 0
test_df.loc[:, 'chid_ratio_bool'] = 0
testing_private_df.loc[:, 'chid_ratio_bool'] = 0
training_df.loc[training_df['chid_ratio'] >= 90, 'chid_ratio_bool'] = 1
test_df.loc[test_df['chid_ratio'] >= 90, 'chid_ratio_bool'] = 1
testing_private_df.loc[testing_private_df['chid_ratio'] >= 90, 'chid_ratio_bool'] = 1
training_df[ 'chid_ratio_bool'].astype("category")
test_df[ 'chid_ratio_bool'].astype("category")
testing_private_df[ 'chid_ratio_bool'].astype("category")



# cano101
training_df.loc[:, 'cano101'] = 0
test_df.loc[:, 'cano101'] = 0
testing_private_df.loc[:, 'cano101'] = 0
training_df.loc[training_df['cano_ratio'] >= 99, 'cano101'] = 1
test_df.loc[test_df['cano_ratio'] >= 99, 'cano101'] = 1
testing_private_df.loc[testing_private_df['cano_ratio'] >= 99, 'cano101'] = 1
training_df.loc[training_df['cano_ratio'] <= 1, 'cano101'] = -1
test_df.loc[test_df['cano_ratio'] <= 1, 'cano101'] = -1
testing_private_df.loc[testing_private_df['cano_ratio'] <= 1, 'cano101'] = -1
training_df[ 'cano101'].astype("category")
test_df[ 'cano101'].astype("category")
testing_private_df[ 'cano101'].astype("category")

# chid101
training_df.loc[:, 'chid101'] = 0
test_df.loc[:, 'chid101'] = 0
testing_private_df.loc[:, 'chid101'] = 0
training_df.loc[training_df['chid_ratio'] >= 99, 'chid101'] = 1
test_df.loc[test_df['chid_ratio'] >= 99, 'chid101'] = 1
testing_private_df.loc[testing_private_df['chid_ratio'] >= 99, 'chid101'] = 1
training_df.loc[training_df['chid_ratio'] <= 1, 'chid101'] = -1
test_df.loc[test_df['chid_ratio'] <= 1, 'chid101'] = -1
testing_private_df.loc[testing_private_df['chid_ratio'] <= 1, 'chid101'] = -1
training_df[ 'chid101'].astype("category")
test_df[ 'chid101'].astype("category")
testing_private_df[ 'chid101'].astype("category")




C:\Users\lo\AppData\Local\Temp\ipykernel_22796\3212495459.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_df.loc[:, 'cano_ratio_bool'] = 0
C:\Users\lo\AppData\Local\Temp\ipykernel_22796\3212495459.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.loc[:, 'cano_ratio_bool'] = 0
C:\Users\lo\AppData\Local\Temp\ipykernel_22796\3212495459.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


0        -1
1        -1
2        -1
3        -1
4        -1
         ..
754134   -1
754135   -1
754136   -1
754137   -1
754138   -1
Name: chid101, Length: 754139, dtype: category
Categories (3, int64): [-1, 0, 1]

In [96]:
# display(training_df[training_df['cano_ratio'].notna()])
# #fill na with 0
# # training_df['cano_ratio'].fillna(0, inplace=True)
# display(training_df[training_df['cano_ratio'].notna()])


# training_df.drop(['chid', 'cano'], axis=1, inplace=True)
# test_df.drop(['chid', 'cano'], axis=1, inplace=True)
# testing_private_df.drop(['chid', 'cano'], axis=1, inplace=True)
training_df.drop(['cano_ratio', 'chid_ratio', 'cano_ratio_bool', 'chid_ratio_bool'], axis=1, inplace=True)
test_df.drop(['cano_ratio', 'chid_ratio', 'cano_ratio_bool', 'chid_ratio_bool'], axis=1, inplace=True)
testing_private_df.drop(['cano_ratio', 'chid_ratio', 'cano_ratio_bool', 'chid_ratio_bool'], axis=1, inplace=True)

display(training_df)
display(test_df)
display(testing_private_df)


# save new feature into temp_training and temp_testing and temp_prediction
# temp_training['cano_ratio'] = training_df['cano_ratio']
# temp_testing['cano_ratio'] = test_df['cano_ratio']
# temp_prediction['cano_ratio'] = testing_private_df['cano_ratio']




C:\Users\lo\AppData\Local\Temp\ipykernel_22796\1089620814.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_df.drop(['cano_ratio', 'chid_ratio', 'cano_ratio_bool', 'chid_ratio_bool'], axis=1, inplace=True)
C:\Users\lo\AppData\Local\Temp\ipykernel_22796\1089620814.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop(['cano_ratio', 'chid_ratio', 'cano_ratio_bool', 'chid_ratio_bool'], axis=1, inplace=True)
C:\Users\lo\AppData\Local\Temp\ipykernel_22796\1089620814.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation:

,chid,cano,mchno,acqic,mcc,conam,flam1,stocn,scity,stscd,hcefg,csmcu,cano101,chid101
0,256109,154065,138119,4871,351,1524.0,1529,0,11874,5,6,68,-1,-1
1,297980,350989,45877,1881,258,101.0,101,0,11874,5,8,68,-1,-1
2,425640,76572,62433,4871,264,117.0,116,0,11874,5,6,68,-1,-1
3,168531,167846,138119,1844,351,1676.0,1683,0,11874,5,6,68,-1,-1
4,493852,407593,55077,1881,382,14.0,14,0,11874,5,6,68,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688521,325181,174672,41109,3193,258,49.0,49,0,11874,5,0,68,-1,-1
8688522,439282,104273,167076,4871,252,4020.0,4018,0,11874,5,6,68,-1,-1
8688523,257694,103443,65068,1613,248,315.0,314,0,8472,5,6,68,-1,-1
8688524,57843,630419,136008,1881,248,226.0,225,0,11874,5,6,68,-1,-1


,chid,cano,mchno,acqic,mcc,conam,flam1,stocn,scity,stscd,hcefg,csmcu,cano101,chid101
0,414123,389764,95079,1425,248,1353.00,1349,0,11874,5,6,68,-1,-1
1,350019,396507,150662,2409,296,305.00,306,0,10068,5,6,68,-1,-1
2,273500,168466,105697,5242,466,20.00,20,0,11874,5,11,82,-1,-1
3,481778,381976,128957,3802,248,8371.00,8339,40,9251,5,6,68,-1,-1
4,77056,294786,14552,4659,194,163.69,163,120,11109,5,6,66,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600177,74460,84881,145268,664,208,96.00,96,0,12143,5,6,68,-1,-1
600178,427331,34935,144308,1881,351,2014.00,2017,0,11874,5,6,68,-1,-1
600179,153100,178100,47446,4456,382,33.00,33,120,7624,5,6,68,-1,-1
600180,389252,195847,1484,4871,264,1800.00,1796,0,11874,5,6,68,-1,-1


,chid,cano,mchno,acqic,mcc,conam,flam1,stocn,scity,stscd,hcefg,csmcu,cano101,chid101
0,484998,269010,141954,4871,264,100.0,100,0,11874,5,6,68,-1,-1
1,45092,197043,158522,1881,252,222.0,222,0,11874,5,6,68,-1,-1
2,209476,539214,145268,4504,208,259.0,260,0,12143,5,6,68,-1,-1
3,56593,619224,13085,8077,324,3111.0,3098,0,11874,5,6,68,-1,-1
4,419897,433534,59776,4871,265,992.0,996,0,11874,5,6,68,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754134,103615,54550,49315,2409,296,209.0,209,0,12142,5,6,68,-1,-1
754135,232916,39574,79991,1881,252,189.0,188,0,11874,5,6,68,-1,-1
754136,82755,308568,99169,6360,248,9100.0,9091,0,11874,5,0,68,-1,-1
754137,406705,323914,49315,2409,296,86.0,86,0,12142,5,6,68,-1,-1


Delete features.

In [91]:
#!!!
#drop columns : csmam, loctm, locdt, contp, etymd, ecfg, insfg, iterm, bnsfg, ovrlt, flbmk, flg_3dsmk
#deal training data 
training_df.drop(['csmam','loctm','locdt','contp','etymd','ecfg','insfg','iterm','bnsfg','ovrlt','flbmk','flg_3dsmk'], axis=1, inplace=True)
test_df.drop(['csmam','loctm','locdt','contp','etymd','ecfg','insfg','iterm','bnsfg','ovrlt','flbmk','flg_3dsmk'], axis=1, inplace=True)
testing_private_df.drop(['csmam','loctm','locdt','contp','etymd','ecfg','insfg','iterm','bnsfg','ovrlt','flbmk','flg_3dsmk'], axis=1, inplace=True)


C:\Users\lo\AppData\Local\Temp\ipykernel_22796\35890505.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_df.drop(['csmam','loctm','locdt','contp','etymd','ecfg','insfg','iterm','bnsfg','ovrlt','flbmk','flg_3dsmk'], axis=1, inplace=True)
C:\Users\lo\AppData\Local\Temp\ipykernel_22796\35890505.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop(['csmam','loctm','locdt','contp','etymd','ecfg','insfg','iterm','bnsfg','ovrlt','flbmk','flg_3dsmk'], axis=1, inplace=True)
C:\Users\lo\AppData\Local\Temp\ipykernel_22796\35890505.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

In [109]:
#drop columns : acqic stocn csmcu stscd flam1 hcefg mcc

#deal training data 
training_df.drop(['acqic' ,'stocn' ,'csmcu', 'stscd', 'flam1' ,'hcefg', 'mcc'], axis=1, inplace=True)

#deal testing data
test_df.drop(['acqic' ,'stocn' ,'csmcu', 'stscd', 'flam1' ,'hcefg', 'mcc'], axis=1, inplace=True)


In [31]:
#deal training data 
# training_df.drop(['cano_ratio' ], axis=1, inplace=True)
# training_df.drop(['cano_ratio_bool' ], axis=1, inplace=True)

#deal testing data
# test_df.drop(['cano_ratio' ], axis=1, inplace=True)
# test_df.drop(['chid_ratio' ], axis=1, inplace=True)
# test_df.drop(['cano_ratio_bool' ], axis=1, inplace=True)




display(training_df.head())
display(test_df.head())





C:\Users\lo\AppData\Local\Temp\ipykernel_22796\2282137337.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop(['cano_ratio' ], axis=1, inplace=True)
C:\Users\lo\AppData\Local\Temp\ipykernel_22796\2282137337.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop(['chid_ratio' ], axis=1, inplace=True)


,chid,cano,mchno,acqic,mcc,conam,flam1,stocn,scity,stscd,hcefg,csmcu,cano_ratio_bool,chid_ratio_bool
0,252906,151533,133492,4783,342,1524.0,1529,0,11343,5,6,68,0,0
1,294266,345075,44365,1850,249,101.0,101,0,11343,5,8,68,0,0
2,420351,75333,60332,4783,255,117.0,116,0,11343,5,6,68,0,0
3,166419,165070,133492,1813,342,1676.0,1683,0,11343,5,6,68,0,0
4,487752,400729,53250,1850,373,14.0,14,0,11343,5,6,68,0,0


,chid,cano,mchno,acqic,mcc,conam,flam1,stocn,scity,stscd,hcefg,csmcu,cano_ratio_bool,chid_ratio_bool
0,408959,383210,91878,1399,239,1353.00,1349,0,11343,5,6,68,0,0
1,345627,389834,145602,2371,287,305.00,306,0,9615,5,6,68,0,0
2,270090,165681,102142,5140,457,20.00,20,0,11343,5,11,82,0,0
3,475823,375542,124619,3734,239,8371.00,8339,39,8828,5,6,68,0,0
4,76056,289821,14071,4577,185,163.69,163,118,10606,5,6,66,0,0


In [97]:
#!!!
# y = training_df['label']
# X = training_df.drop(['label','txkey'], axis=1)
# training_df = training_df.drop(['cano_ratio'])
X = training_df
#X randomly the rows
# X = X.sample(n=X.shape[0], random_state=42, replace=False)
# display(X)
# display(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)



## training & testing.
### LightGBM

In [18]:



# df_sample = df.sample(n=300000, random_state=42)
# train_df, val_df = train_test_split(df_sample, test_size=0.2, random_state=42)

# # Define the features and target variable
# #featues 前面preprocess先做好這邊在註解掉
# features = [ 'locdt','loctm','contp','etymd','mcc','conam','ecfg','insfg','iterm','bnsfg','flam1','stocn','scity','ovrlt','flbmk','hcefg','csmcu','csmam','flg_3dsmk'] 
# target = 'label'

# # Split the training and validation datasets into features and target variable
# X_train = train_df[features]
# y_train = train_df[target]
# X_val = val_df[features]
# y_val = val_df[target]

# # Train the model using LightGBM

# # params = {
# #     'objective': 'binary',
# #     'metric': 'binary_logloss',
# #     'boosting_type': 'dart',
# #     'num_leaves': 50,  
# #     'learning_rate': 0.01,  
# #     'feature_fraction': 0.8  
# # }
# params = {'objective': 'binary',
#           'boosting_type': 'gbdt',
#           'metric': 'auc',
#           'learning_rate': 0.007,
#           'num_leaves': 2**8,
#           'max_depth': -1,
#           'tree_learner':'serial',
#           'colsample_bytree': 0.5,
#           'subsample_freq':1,
#           'subsample':0.7,
#           'n_estimators':10000,
#           'min_data_in_leaf': 106,
#           'max_bin':255,
#           'verbosity': -1,
#           'early_stopping_rounds':100,
#          }


# train_data = lgb.Dataset(X_train, label=y_train)
# val_data = lgb.Dataset(X_val, label=y_val)

# model = lgb.train(params, train_data, num_boost_round=10000, valid_sets=[train_data, val_data])
# model = lgb.train(params, train_data, num_boost_round=1000, valid_sets=[train_data, val_data], early_stopping_rounds=50)

categories = ['chid', 'cano', 'contp', 'etymd', 'mchno', 'acqic',
              'mcc',  'ecfg', 'insfg',  'bnsfg', 'stocn', 'scity',
              'stscd', 'ovrlt', 'flbmk', 'hcefg', 'csmcu', 'flg_3dsmk']

params = {
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 60,
    'max_depth': 6,
    'learning_rate': 0.005,
    'n_estimators': 3000,
    'boosting_type': 'gbdt',
}   

train_data = lgb.Dataset(X_train, label=y_train,
                         categorical_feature=categories)
test_data = lgb.Dataset(X_test, label=y_test,
                        reference=train_data)

bst = lgb.train(params, train_data,num_boost_round = 100,  valid_sets=[test_data],categorical_feature=categories)

/home/swc/DM_Final/lib/python3.8/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 25694, number of negative: 6925126
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.216511 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 39625
[LightGBM] [Info] Number of data points in the train set: 6950820, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.003697 -> initscore=-5.596654
[LightGBM] [Info] Start training from score -5.596654
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

validation.

In [27]:
y_pred_prob = bst.predict(X_test, num_iteration=bst.best_iteration)
y_pred2 = [1 if x >= 0.5 else 0 for x in y_pred_prob]  # Assuming binary classification, adjust as needed
# y_pred = model.predict(X_val)
# print(y_pred)
# # Get the true labels from the validation set
# y_true = val_data.label

# # Calculate F1 score
# f1 = f1_score(y_val, y_pred)
# print(f'F1 Score on Validation Set: {f1}')

# y_pred = pd.Series(bst.predict(X_test, num_iteration=bst.best_iteration)>0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred2)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred2,digits=5))

Accuracy: 0.9977740768576503
Classification Report:
              precision    recall  f1-score   support

           0    0.99811   0.99966   0.99888   1731371
           1    0.83785   0.48287   0.61266      6335

    accuracy                        0.99777   1737706
   macro avg    0.91798   0.74127   0.80577   1737706
weighted avg    0.99753   0.99777   0.99748   1737706



### XGboost

In [98]:
#!!!
from xgboost import XGBClassifier
xgboostModel = XGBClassifier( n_estimators=12000, learning_rate= 0.005,enable_categorical=True,
                             objective = "binary:hinge",eval_metric="auc",max_depth=12, device="cuda")#grow_policy="lossguide"
xgboostModel.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=True, eval_metric='auc', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.005, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=12, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=12000, n_jobs=None,
              num_parallel_tree=None, objective='binary:hinge', ...)

In [49]:
y_pred = xgboostModel.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred,digits=5))

Accuracy: 0.999858434050409
Classification Report:
              precision    recall  f1-score   support

           0    0.99991   0.99995   0.99993    865674
           1    0.98570   0.97546   0.98055      3179

    accuracy                        0.99986    868853
   macro avg    0.99280   0.98771   0.99024    868853
weighted avg    0.99986   0.99986   0.99986    868853



## predict

Use pubic data to validation and test the f1-score.

In [17]:
#!!!
label_encoder_txkey = LabelEncoder()
test_allData = pd.concat([testing_df ,testing_answer_df])
testing_num = testing_df.shape[0]
test_allData['txkey'] = label_encoder_txkey.fit_transform(test_allData['txkey'])
the_testing_df = test_allData[:testing_num]
the_testing_answer_df = test_allData[testing_num:]
# Use map() function to find answer in testing_answer_df using txkey label
ground_truth = []
ground_truth = the_testing_df['txkey'].map(the_testing_answer_df.set_index('txkey')['label'])
ground_truth = ground_truth.tolist()
ground_truth = np.array(ground_truth)



In [99]:
#!!!
#predict test_df than use testing_df's txkey to get the answer in testing_answer_df
#use xgboostModel to training
display(test_df)
display(X_train)
# display(ground_truth.nonzero()[0].shape[0])

# switch test_df's column : cano_ratio_bool and cano_ratio 
# temp = test_df['cano_ratio_bool']
# test_df['cano_ratio_bool'] = test_df['cano_ratio']
# test_df['cano_ratio'] = temp
#also swtcich catergory name


y_pred = xgboostModel.predict(test_df)
# display(test_df.head())
#use bst to training
# y_pred_prob = bst.predict(test_df, num_iteration=bst.best_iteration)
# y_pred = [1 if x >= 0.5 else 0 for x in y_pred_prob]  
#calculate f1 score
f1 = f1_score(ground_truth, y_pred)
print(f'F1 Score on Public dataset: {f1}')


,chid,cano,mchno,acqic,mcc,conam,flam1,stocn,scity,stscd,hcefg,csmcu,cano101,chid101
0,414123,389764,95079,1425,248,1353.00,1349,0,11874,5,6,68,-1,-1
1,350019,396507,150662,2409,296,305.00,306,0,10068,5,6,68,-1,-1
2,273500,168466,105697,5242,466,20.00,20,0,11874,5,11,82,-1,-1
3,481778,381976,128957,3802,248,8371.00,8339,40,9251,5,6,68,-1,-1
4,77056,294786,14552,4659,194,163.69,163,120,11109,5,6,66,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600177,74460,84881,145268,664,208,96.00,96,0,12143,5,6,68,-1,-1
600178,427331,34935,144308,1881,351,2014.00,2017,0,11874,5,6,68,-1,-1
600179,153100,178100,47446,4456,382,33.00,33,120,7624,5,6,68,-1,-1
600180,389252,195847,1484,4871,264,1800.00,1796,0,11874,5,6,68,-1,-1


,chid,cano,mchno,acqic,mcc,conam,flam1,stocn,scity,stscd,hcefg,csmcu,cano101,chid101
4160620,115668,100672,15592,6360,425,105.0,105,0,11874,5,6,68,-1,-1
7447568,226853,145955,146709,1881,382,1.0,1,0,11874,5,6,68,-1,-1
4982111,317529,153258,127767,8580,299,66.0,66,51,8300,5,6,68,-1,-1
3414894,385960,232820,140509,1425,248,706.0,705,0,11875,5,6,68,-1,-1
8259103,364041,36167,49382,3690,435,590.0,590,0,11915,5,6,68,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7204212,119863,233944,105697,5242,465,400.0,398,0,11874,5,11,82,-1,-1
2234489,408731,397044,14927,1881,258,280.0,279,0,11874,5,0,68,-1,-1
4304572,449816,231179,66325,6454,192,845.0,843,0,11874,5,6,68,-1,-1
6550634,18264,206039,135884,6360,296,550.0,548,0,11874,5,6,68,-1,-1


F1 Score on Public dataset: 0.4307909604519774


Predict part.

In [70]:
#use xgboostModel to training
y_pred = xgboostModel.predict(testing_private_df)

# #use bst to training
# predict_data = testing_private_df
# y_pred = pd.Series(bst.predict(predict_data, num_iteration=bst.best_iteration)>0.5).astype(int)


In [88]:
# test_df = pd.read_csv(test_file_path)
# pridict = pd.DataFrame(testing_private_df['txkey'] ) # go up to before drop txkey
# pridict['pred'] = y_pred 

# display(y_pred )
# print(np.nonzero(y_pred))
# print(y_pred[1771])
# display(pridict.iloc[1900])
# display(pridict)
a = pd.DataFrame(testing_df['txkey'] ) # go up to before drop txkey
a['pred'] = ground_truth.astype(int)
output = pd.concat([a, pridict], axis=0)
display(output)

,txkey,pred
0,a2c1209018e4e52e04f6fabb48f05f1b8bc09dc838ff6c...,0
1,16c4880500059e01553789be11bbb50753b7acaae7b95b...,0
2,623c56be3bee87724e3d119c271d9ed098eeda84233183...,0
3,250da12187059cf6e3a3066656a2919d08ceb8207efd55...,0
4,4b268e0da036f44cbbb056ddfac6a28ea336d9cf299843...,0
...,...,...
754134,47cd55bd7492e01e197b61799d072e6ca9e1c118bc76e9...,0
754135,26cef9cc35fb73bedd08a711ebb6d612e4af7dd7b62e7c...,0
754136,f12f861ef6a8517ecddae262c1cc895f618e7e8c19eb9a...,0
754137,638b1a6a6f4389b7af60a6cb0ca908c82d0bdffc6924b0...,0


## output

In [89]:
output.to_csv('..\\output.csv', index=False)
# output.to_csv('output.csv', index=False)
